In [21]:
# mypy: disable-error-code="no-untyped-def, no-untyped-call, assignment"
# pyright: reportUnusedExpression=false

# Why `cmomy`?

$$\newcommand{\ave}[1]{\langle{#1}\rangle}$$

We wish to calculate moments of the form $\ave{x}$ and $\ave{(\delta x)^k}$, $\delta x = (x - \ave{x})$, where, in general we define averages
of the form.

$$
   \ave{x} = \frac{\sum_i w_i x_i}{\sum_i w_i}
$$
$$
  \ave{(\delta x)^k} = \frac{\sum_i w_i (x_i - \ave{x})^k}{\sum_i w_i}
$$

where $w_i$ are weights associated with each sample $x_i$.  In the simple case, $w_i = 1$.  To calculate a central moment with $k > 1$, we have some choices.  

1. First calculate $\ave{x}$, then use the above formula.
* Downside: requires two passes through the data.  
* Upside: numerically stable.
    
2. Expand out the central moment to raw moments.  For example $\ave{(x - \ave{x})^2} = \ave{x^n} - \ave{x}^2$.  These raw moments can be calculated directly in one pass.  
* Downside: numerically unstable ([see here for some info](https://en.wikipedia.org/wiki/Algorithms_for_calculating_variance)). 
* Upside: single pass for each of $\ave{x^k}$.


## An example

    

In what follows, we'll arrange data in the from `output[moment]` where `output[0] = {total weight}`, `output[1] = {average value}`, `output[k > 1] = {kth order central moment}`.  We can generate the central moments using the following functions

In [1]:
def cmom_1(x, axis=0, mom=3):
    """Create central moments using stable method"""
    # output shape
    shape = x.shape[:axis] + x.shape[axis + 1 :] + (mom + 1,)
    output = np.zeros(shape, dtype=x.dtype)

    output[..., 0] = x.shape[axis]
    output[..., 1] = x.mean(axis=axis)

    # moments [2 -> mom]
    output[..., 2:] = (
        (x - x.mean(axis=axis, keepdims=True))[..., None] ** np.arange(2, mom + 1)
    ).mean(axis=axis)
    return output


def cmom_2(x, axis=0, mom=3):
    if mom > 3:
        raise ValueError

    shape = x.shape[:axis] + x.shape[axis + 1 :] + (mom + 1,)

    output = np.zeros(shape, dtype=x.dtype)

    output[..., 0] = x.shape[axis]

    # higher order raw moments
    raws = (x[..., None] ** np.arange(0, mom + 1)).mean(axis=axis)

    output[..., 1] = raws[..., 1]
    if mom > 1:
        # <(x - <x>)**2> = <x**2> - <x>**2
        output[..., 2] = raws[..., 2] - raws[..., 1] ** 2

    if mom > 2:
        # <(x - <x>)**3> = <x**3> - 3<x**2><x> + 2<x>**3
        output[..., 3] = (
            raws[..., 3] - 3 * raws[..., 2] * raws[..., 1] + 2 * raws[..., 1] ** 3
        )

    return output

In [2]:
import numpy as np

np.random.seed(0)
n = 1000
x = np.random.rand(n)

m1 = cmom_1(x)
m2 = cmom_2(x)
# same values
np.testing.assert_allclose(m1, m2)

In [3]:
np.set_printoptions(precision=4)

OK, great.  But what if we have unscaled data?  For example, setting $y = a x + b$ gives $\ave{y} = a \ave{x} + b$, but for central moments, $\ave{(y - \ave{y})^n} = \ave{(a x - b - \ave{a x - b})^n} = a^n \ave{(\delta x)^n}$

In [4]:
# large value, small fluctuation
a = 0.1
b = 1e4
y = a * x + b

# expected value after shift
expected = m1.copy()
expected[..., 1] = a * expected[..., 1] + b
expected[..., 2:] = expected[..., 2:] * (a ** np.arange(2, 4))


m1_shift = cmom_1(y)
m2_shift = cmom_2(y)

m1_error = m1_shift - expected
m2_error = m2_shift - expected

print(
    f"""
abs error using central moments: {m1_error}
rel error using central moments: {m1_error / expected}
abs error using raw moments    : {m2_error}
rel error using raw moments    : {m2_error / expected}
"""
)


abs error using central moments: [0.0000e+00 0.0000e+00 3.9844e-16 1.4604e-15]
rel error using central moments: [0.0000e+00 0.0000e+00 4.7182e-13 9.3328e-10]
abs error using raw moments    : [ 0.0000e+00  1.8190e-12  4.6443e-08 -2.6871e-03]
rel error using raw moments    : [ 0.0000e+00  1.8190e-16  5.4997e-05 -1.7173e+03]



## So what?

OK, so central moments have some advantages.  But the two pass thing makes life difficult.  For example, what if we are running a slow experiment or simulation?  We could collect all the samples for `x`, then do the two pass algorithm at the end.  But it would be nice to calculate averages on the fly.  This is why we often turn to raw moments.  There easy to accumulate as data comes in, while central moments, at least as formulated above, are not.  

Thankfully, smart people have figured out how to calculate central moments in a one-pass way.  Moreover, these methods allow central moments to be combined.  This makes resampling from central moments possible.  We don't go into detail on these formulas, but point the interested reader to the this [article](https://en.wikipedia.org/wiki/Algorithms_for_calculating_variance), and citations within.

The `cmomy` package has a variety of routines to work with central moments.  This include

* Create central moments accumulator object in a variety of ways.
* add data to object
* resample data
* extract central and raw moments


The primary object for doing all this is {class}`~cmomy.CentralMoments`.

In [8]:
import cmomy

cx = cmomy.CentralMoments.from_vals(x, axis=0, mom=3)
cy = cmomy.CentralMoments.from_vals(y, axis=0, mom=3)


cx

<CentralMoments(val_shape=(), mom=(3,))>
array([1.0000e+03, 4.9592e-01, 8.4448e-02, 1.5648e-03])

You can access the underlying data using the {attr}`cmomy.CentralMoments.values` or {attr}`cmomy.CentralMoments.data` attributes.

In [9]:
print(cx.values)

[1.0000e+03 4.9592e-01 8.4448e-02 1.5648e-03]


Basically, the {class}`~cmomy.CentralMoments` object is just a collection of routines around {attr}`~cmomy.CentralMoments.data`, which is an array of central moments (with the convention that `data[..., 0]` is the total `weight`, `data[..., 1]` is the average, and `data[..., k > 1]` is the `kth` central moment).  It gives the same results as calculating central moments directly:

In [8]:
np.testing.assert_allclose(cx.values, m1)
np.testing.assert_allclose(cy.values, m1_shift)

Nice!  But what can that's not all that special.  The real power comes in for multidimensional data.
Suppose that we have separate samples of the data $\{x_a, x_b, ... \}$, and want to get there central moments.  We can do the following.

In [9]:
x = np.random.rand(100, 5)
m1 = cmom_1(x, axis=0, mom=4)
m1

array([[ 1.0000e+02,  4.7485e-01,  8.4724e-02,  2.7697e-03,  1.3407e-02],
       [ 1.0000e+02,  4.8072e-01,  8.7882e-02,  8.1934e-04,  1.3999e-02],
       [ 1.0000e+02,  4.9077e-01,  9.2369e-02,  5.5313e-03,  1.5294e-02],
       [ 1.0000e+02,  5.4378e-01,  1.0249e-01, -5.3586e-03,  1.6396e-02],
       [ 1.0000e+02,  5.6677e-01,  8.0621e-02, -8.4636e-03,  1.3304e-02]])

Wrapping the data in a central moments object gives

In [10]:
c = cmomy.CentralMoments.from_data(m1, mom_ndim=1)

# reduce the data along a dimension to get

print(c.reduce(axis=0).values)

# verify that this is the same as just accumulating all the data
print(cmom_1(x.reshape(-1), mom=4))

[ 5.0000e+02  5.1138e-01  9.0981e-02 -8.6037e-04  1.4484e-02]
[ 5.0000e+02  5.1138e-01  9.0981e-02 -8.6037e-04  1.4484e-02]


Excellent! We can use {class}`~cmomy.CentralMoments` to accumulate data on the go

In [11]:
c = cmomy.CentralMoments.zeros(mom=4)
for i in range(5):
    # note: pushing multiple values
    c.push_vals(x[:, i])
c

<CentralMoments(val_shape=(), mom=(4,))>
array([ 5.0000e+02,  5.1138e-01,  9.0981e-02, -8.6037e-04,  1.4484e-02])

Or, you can push all the values one at a time!

In [12]:
c.zero()
for xx in x.reshape(-1):
    # note: pushing single value
    c.push_val(xx)

c

<CentralMoments(val_shape=(), mom=(4,))>
array([ 5.0000e+02,  5.1138e-01,  9.0981e-02, -8.6037e-04,  1.4484e-02])

The values can even be differently weighted.

In [13]:
xs = np.split(x.reshape(-1), [50, 150, 300])

c.zero()
for v in xs:
    c.push_vals(v)
c

<CentralMoments(val_shape=(), mom=(4,))>
array([ 5.0000e+02,  5.1138e-01,  9.0981e-02, -8.6037e-04,  1.4484e-02])

Alternatively, you can add objects together.

In [14]:
cs = [cmomy.CentralMoments.from_vals(v, mom=4) for v in xs]
print(cs)

c_tot = cs[0] + cs[1] + cs[2] + cs[3]
c_tot

[<CentralMoments(val_shape=(), mom=(4,))>
array([5.0000e+01, 4.8322e-01, 8.9160e-02, 1.7039e-03, 1.4934e-02]), <CentralMoments(val_shape=(), mom=(4,))>
array([1.0000e+02, 4.8959e-01, 9.6020e-02, 1.9161e-03, 1.5975e-02]), <CentralMoments(val_shape=(), mom=(4,))>
array([ 1.5000e+02,  5.2217e-01,  9.1658e-02, -1.9119e-03,  1.4501e-02]), <CentralMoments(val_shape=(), mom=(4,))>
array([ 2.0000e+02,  5.2121e-01,  8.7789e-02, -1.8882e-03,  1.3540e-02])]


<CentralMoments(val_shape=(), mom=(4,))>
array([ 5.0000e+02,  5.1138e-01,  9.0981e-02, -8.6037e-04,  1.4484e-02])

Or more simply:

In [15]:
print(sum(cs, start=cs[0].zeros_like()).values)

[ 5.0000e+02  5.1138e-01  9.0981e-02 -8.6037e-04  1.4484e-02]


Very cool!

## weighted averages

cmomy is setup to work with arbitrary weights.  We saw this work above when we considered unequal sample sizes.  For example,

In [16]:
def get_cmom_with_weights(x, w, axis=0, mom=3):
    """Create central moments using stable method"""
    # output shape
    shape = x.shape[:axis] + x.shape[axis + 1 :] + (mom + 1,)
    output = np.zeros(shape, dtype=x.dtype)

    w_sum = w.sum(axis=axis)
    w_norm = 1.0 / w_sum

    output[..., 0] = w_sum
    output[..., 1] = ((w * x).sum(axis=axis)) * w_norm

    # moments [2 -> mom]
    output[..., 2:] = (
        w[..., None]
        * (x - (w * x).sum(axis=axis, keepdims=True) * w_norm)[..., None]
        ** np.arange(2, mom + 1)
    ).sum(axis=axis) * w_norm

    return output

In [17]:
x = np.random.rand(100)
w = np.random.rand(100)

moments = get_cmom_with_weights(x, w)
moments

cmomy.CentralMoments.from_vals(x, w=w, mom=3)

<CentralMoments(val_shape=(), mom=(3,))>
array([ 5.1337e+01,  5.4415e-01,  8.9238e-02, -6.9704e-03])

## comoments

cmomy can also handle comoments (covariance, etc), like the $\ave{(\delta x)^i (\delta y)^j}$    
For example

In [18]:
y = np.random.rand(100)

cmomy.CentralMoments.from_vals(x=(x, y), w=w, mom=(2, 2))

<CentralMoments(val_shape=(), mom=(2, 2))>
array([[ 5.1337e+01,  4.7125e-01,  9.1568e-02],
       [ 5.4415e-01,  1.8065e-02, -2.0110e-03],
       [ 8.9238e-02, -3.5802e-03,  7.0305e-03]])

All the special sauce works for central moments or comoments.  

## xarray DataArray support

cmomy also works with {class}`xarray.DataArray` objects.  For example

In [19]:
import xarray as xr

x = xr.DataArray(np.random.rand(100, 2, 3), dims=["rec", "a", "b"])
x.head(3)  # type: ignore[attr-defined]

<xarray.DataArray (rec: 3, a: 2, b: 3)>
array([[[0.547 , 0.2936, 0.9682],
        [0.2262, 0.0157, 0.3259]],

       [[0.5025, 0.0284, 0.5592],
        [0.8743, 0.7047, 0.623 ]],

       [[0.956 , 0.9583, 0.8243],
        [0.6077, 0.4878, 0.0133]]])
Dimensions without coordinates: rec, a, b

To create a cmomy object wrapping a {class}`xarray.DataArray`, use {class}`~cmomy.xCentralMoments`

In [20]:
c = cmomy.xCentralMoments.from_vals(x, dim="rec", mom=3)
c

<xCentralMoments(val_shape=(2, 3), mom=(3,))>
<xarray.DataArray (a: 2, b: 3, mom_0: 4)>
array([[[ 1.0000e+02,  5.3672e-01,  8.3228e-02, -2.2716e-03],
        [ 1.0000e+02,  4.9449e-01,  7.0698e-02,  2.5130e-03],
        [ 1.0000e+02,  5.4783e-01,  7.4566e-02, -6.1021e-03]],

       [[ 1.0000e+02,  5.0708e-01,  8.7601e-02, -8.6529e-04],
        [ 1.0000e+02,  4.6023e-01,  9.3925e-02,  5.8270e-03],
        [ 1.0000e+02,  4.7648e-01,  7.9938e-02,  5.5096e-03]]])
Dimensions without coordinates: a, b, mom_0

Everything that {class}`~cmomy.CentralMoments` can do, {class}`~cmomy.xCentralMoments` can do.